In [339]:
# import libraties
import pandas as pd
import numpy as np
import json
import pickle

# https://pm4py.fit.fraunhofer.de/documentation
import pm4py
from pm4py.objects.log.obj import EventLog, Trace
from pm4py.objects.log.util.log import project_traces
from pm4py.objects.log.util import interval_lifecycle
from pm4py.statistics.sojourn_time.log import get as soj_time_get
from pm4py.algo.transformation.log_to_features import algorithm as log_to_features

import warnings
warnings.filterwarnings("ignore")

In [340]:
# function to project the trace
def project_nth(log, index):
    print(str(project_traces(log)[index]))

In [341]:
# read data in csv 
trace = pd.read_csv('../data/BPIC2020_CSV/filterd_travel_permit_final.csv')
trace.head()

# to read from xes file
# data_path = '../data/Travel Permits Filtered.xes'
# trace_log = pm4py.read_xes(data_path)
# trace_log = pm4py.format_dataframe(trace_log, case_id='case:id', activity_key='concept:name', timestamp_key='time:timestamp')
# trace_log = pm4py.convert_to_event_log(trace_log)
# trace_log

,case,event,startTime,completeTime,dec_id_5,dec_id_6,dec_id_3,dec_id_4,dec_id_1,dec_id_2,...,Task_6,org:resource,event_id,org:role,concept:name,time:timestamp,@@index,@@case_index,start_timestamp,case:concept:name
0,travel permit 10022,Permit SUBMITTED by EMPLOYEE,2018-02-20 13:51:27+00:00,2018-02-20 13:51:27+00:00,NaN,NaN,NaN,NaN,declaration 10026,NaN,...,NaN,STAFF MEMBER,st_step 10031_0,EMPLOYEE,Permit SUBMITTED by EMPLOYEE,2018-02-20 13:51:27+00:00,0,0,2018-02-20 13:51:27+00:00,travel permit 10022
1,travel permit 10022,Permit APPROVED by ADMINISTRATION,2018-02-20 13:51:34+00:00,2018-02-20 13:51:34+00:00,NaN,NaN,NaN,NaN,declaration 10026,NaN,...,NaN,STAFF MEMBER,st_step 10030_0,ADMINISTRATION,Permit APPROVED by ADMINISTRATION,2018-02-20 13:51:34+00:00,1,0,2018-02-20 13:51:34+00:00,travel permit 10022
2,travel permit 10022,Permit APPROVED by BUDGET OWNER,2018-02-20 16:27:33+00:00,2018-02-20 16:27:33+00:00,NaN,NaN,NaN,NaN,declaration 10026,NaN,...,NaN,STAFF MEMBER,st_step 10029_0,BUDGET OWNER,Permit APPROVED by BUDGET OWNER,2018-02-20 16:27:33+00:00,2,0,2018-02-20 16:27:33+00:00,travel permit 10022
3,travel permit 10022,Permit FINAL_APPROVED by SUPERVISOR,2018-02-21 12:58:49+00:00,2018-02-21 12:58:49+00:00,NaN,NaN,NaN,NaN,declaration 10026,NaN,...,NaN,STAFF MEMBER,st_step 10028_0,SUPERVISOR,Permit FINAL_APPROVED by SUPERVISOR,2018-02-21 12:58:49+00:00,3,0,2018-02-21 12:58:49+00:00,travel permit 10022
4,travel permit 10022,Start trip,2018-03-17 00:00:00+00:00,2018-03-17 00:00:00+00:00,NaN,NaN,NaN,NaN,declaration 10026,NaN,...,NaN,STAFF MEMBER,rv_travel permit 10022_6,EMPLOYEE,Start trip,2018-03-17 00:00:00+00:00,4,0,2018-03-17 00:00:00+00:00,travel permit 10022


In [342]:
# get persentage of missing values in a column
percent_missing = trace.isnull().sum() * 100 / len(trace)
missing_value_df = pd.DataFrame({'column_name': trace.columns,
                                 'percent_missing': percent_missing})

# extract list of coumns which are more then 30%
miss_ls = list(percent_missing[percent_missing>30].keys())

# keep the remaining columns where missing data is less then 30%
trace = trace.loc[:, ~trace.columns.isin(miss_ls)]
trace.head()

,case,event,startTime,completeTime,id,RequestedAmount_0,Overspent,travel permit number,OverspentAmount,RequestedBudget,...,OrganizationalEntity,org:resource,event_id,org:role,concept:name,time:timestamp,@@index,@@case_index,start_timestamp,case:concept:name
0,travel permit 10022,Permit SUBMITTED by EMPLOYEE,2018-02-20 13:51:27+00:00,2018-02-20 13:51:27+00:00,travel permit 10022,395,False,travel permit number 10023,-50.662542,2134.047941,...,organizational unit 65466,STAFF MEMBER,st_step 10031_0,EMPLOYEE,Permit SUBMITTED by EMPLOYEE,2018-02-20 13:51:27+00:00,0,0,2018-02-20 13:51:27+00:00,travel permit 10022
1,travel permit 10022,Permit APPROVED by ADMINISTRATION,2018-02-20 13:51:34+00:00,2018-02-20 13:51:34+00:00,travel permit 10022,395,False,travel permit number 10023,-50.662542,2134.047941,...,organizational unit 65466,STAFF MEMBER,st_step 10030_0,ADMINISTRATION,Permit APPROVED by ADMINISTRATION,2018-02-20 13:51:34+00:00,1,0,2018-02-20 13:51:34+00:00,travel permit 10022
2,travel permit 10022,Permit APPROVED by BUDGET OWNER,2018-02-20 16:27:33+00:00,2018-02-20 16:27:33+00:00,travel permit 10022,395,False,travel permit number 10023,-50.662542,2134.047941,...,organizational unit 65466,STAFF MEMBER,st_step 10029_0,BUDGET OWNER,Permit APPROVED by BUDGET OWNER,2018-02-20 16:27:33+00:00,2,0,2018-02-20 16:27:33+00:00,travel permit 10022
3,travel permit 10022,Permit FINAL_APPROVED by SUPERVISOR,2018-02-21 12:58:49+00:00,2018-02-21 12:58:49+00:00,travel permit 10022,395,False,travel permit number 10023,-50.662542,2134.047941,...,organizational unit 65466,STAFF MEMBER,st_step 10028_0,SUPERVISOR,Permit FINAL_APPROVED by SUPERVISOR,2018-02-21 12:58:49+00:00,3,0,2018-02-21 12:58:49+00:00,travel permit 10022
4,travel permit 10022,Start trip,2018-03-17 00:00:00+00:00,2018-03-17 00:00:00+00:00,travel permit 10022,395,False,travel permit number 10023,-50.662542,2134.047941,...,organizational unit 65466,STAFF MEMBER,rv_travel permit 10022_6,EMPLOYEE,Start trip,2018-03-17 00:00:00+00:00,4,0,2018-03-17 00:00:00+00:00,travel permit 10022


# train test split

- Do temporal train test split and preprocess them seperately so that no data leakage will be present
- we split data based on 70% 30 % (Temporal split)
- classifier will be trained with all cases that started before a given date T1 which would represent a current point in time in a real-life scenario, and the testing is done only on cases that start afterwards.

In [343]:
trace.head()

,case,event,startTime,completeTime,id,RequestedAmount_0,Overspent,travel permit number,OverspentAmount,RequestedBudget,...,OrganizationalEntity,org:resource,event_id,org:role,concept:name,time:timestamp,@@index,@@case_index,start_timestamp,case:concept:name
0,travel permit 10022,Permit SUBMITTED by EMPLOYEE,2018-02-20 13:51:27+00:00,2018-02-20 13:51:27+00:00,travel permit 10022,395,False,travel permit number 10023,-50.662542,2134.047941,...,organizational unit 65466,STAFF MEMBER,st_step 10031_0,EMPLOYEE,Permit SUBMITTED by EMPLOYEE,2018-02-20 13:51:27+00:00,0,0,2018-02-20 13:51:27+00:00,travel permit 10022
1,travel permit 10022,Permit APPROVED by ADMINISTRATION,2018-02-20 13:51:34+00:00,2018-02-20 13:51:34+00:00,travel permit 10022,395,False,travel permit number 10023,-50.662542,2134.047941,...,organizational unit 65466,STAFF MEMBER,st_step 10030_0,ADMINISTRATION,Permit APPROVED by ADMINISTRATION,2018-02-20 13:51:34+00:00,1,0,2018-02-20 13:51:34+00:00,travel permit 10022
2,travel permit 10022,Permit APPROVED by BUDGET OWNER,2018-02-20 16:27:33+00:00,2018-02-20 16:27:33+00:00,travel permit 10022,395,False,travel permit number 10023,-50.662542,2134.047941,...,organizational unit 65466,STAFF MEMBER,st_step 10029_0,BUDGET OWNER,Permit APPROVED by BUDGET OWNER,2018-02-20 16:27:33+00:00,2,0,2018-02-20 16:27:33+00:00,travel permit 10022
3,travel permit 10022,Permit FINAL_APPROVED by SUPERVISOR,2018-02-21 12:58:49+00:00,2018-02-21 12:58:49+00:00,travel permit 10022,395,False,travel permit number 10023,-50.662542,2134.047941,...,organizational unit 65466,STAFF MEMBER,st_step 10028_0,SUPERVISOR,Permit FINAL_APPROVED by SUPERVISOR,2018-02-21 12:58:49+00:00,3,0,2018-02-21 12:58:49+00:00,travel permit 10022
4,travel permit 10022,Start trip,2018-03-17 00:00:00+00:00,2018-03-17 00:00:00+00:00,travel permit 10022,395,False,travel permit number 10023,-50.662542,2134.047941,...,organizational unit 65466,STAFF MEMBER,rv_travel permit 10022_6,EMPLOYEE,Start trip,2018-03-17 00:00:00+00:00,4,0,2018-03-17 00:00:00+00:00,travel permit 10022


In [344]:
# get completion time and sort
completion_time_ls = list(trace.groupby(['case'])['completeTime'].max())
completion_time_ls = sorted(completion_time_ls)

In [345]:
# split on 70% max time 
split_portion = 0.70
total_data = len(completion_time_ls)
train_len = int(split_portion*total_data)
last_train_completion_time = completion_time_ls[train_len]
last_train_completion_time

'2018-10-15 17:31:12+00:00'

In [346]:
# take all traces where start dates are after the last_train_completion_time
train_df = pd.DataFrame(columns = trace.columns)
test_df = pd.DataFrame(columns = trace.columns)
train_count,test_count = 0,0
for name, group in trace.groupby(['case']):
    if group['startTime'].iloc[0] < last_train_completion_time:
        train_df = train_df.append(group)
        train_count+=1
    else:
        test_df = test_df.append(group)
        test_count+=1

print("train and test count")
print(train_count,test_count)


train and test count
5596 723


In [347]:
# # although the split was 70:30, but since it is sequence data, traces can have different number of data points, so actual number is different 
# test_data_percentage = (test_count/train_count)*100
# print("trace test percentage")
# print(test_data_percentage)

# Define common variables

In [476]:
# trace length and saving path
t_length = 4
save_path_base = '../data/training_data/'

# ====================================
# Preparing Train Data

- define trace length and df_type (train or test)
- Working on train and test seperately, so run whole code for the same trace length for train and then for test
- the repeating code will be modularized when developing python file

In [477]:
# permits is just for the variable name, as below code is using permit as dataframe name
df_type = 'train'
permits = train_df

## Preparing data for task Declaration REJECTED vs Accapted by Administrator/Employee

- keeping all the traces \
or
Extract traces only where decleration is submitted by employee is in a trace ( we are not considering other traces where decision of accatped, rejected is not being made /discuss )
- create target variables, if application is rejected or not
- get traces which starts from permit submitted by employee to the trace decleration is submitted by employee (discuss)
- filter traces of length n (10,15,20 decide) 
- choose encoding and decide features to involve which we belive will be there before the activity decleration is submitted


### Filterings

1. Filtering on timeframe (from this date to that)
2. Filter on case performance (traces finished withing 10 days)
3. Filter on start and end activities (give list of start and end)
4. Filter on variants (keeping only frequent trace flows like - [a,b,c,d] and [a d b c], or 0.4 threshold etc)
5. Filter on attributes values (selection and projection of traces)
6. Filter on numeric attribute values (from declared amount 500 to 1000)
7. Between Filter (filtering the activities from say permit apply to permit accapted)
8. case size (number of activiries in case)


### Statistics

1. Throughput Time (time to complete traces)
2. Case Arrival/Dispersion Ratio (arrival time between two traces (not events))
3. Performance Spectrum (time between activities)
4. Cycle Time and Waiting Time (cycle time - time between activities, lead Time - the overall time in which the instance was worked, from the start to the end,accumelative )
5. Sojourn Time - time taken for an activity to complete from the end of previous activity
6. other stats if needed


# Feature Engineering

### Adding some time features

In [478]:
# adding month and year as feature
permits['startTime'] = pd.to_datetime(permits['startTime'])
permits['completeTime'] = pd.to_datetime(permits['completeTime'])
permits['year'] = permits['startTime'].dt.year
permits['month'] = permits['startTime'].dt.month

In [479]:
# converting dataframe to event log
trace_log = pm4py.format_dataframe(permits, case_id='case', activity_key='event', timestamp_key='completeTime', start_timestamp_key='startTime')
trace_log = pm4py.convert_to_event_log(trace_log)

In [480]:
# adding lead and waiting time
# @@approx_bh_partial_lead_time	- Incremental lead time associated to the event
# @@approx_bh_overall_wasted_time - Difference between the partial lead time and the partial cycle time values (for us same as above, as no cylce time for us)
# @@approx_bh_this_wasted_time -	Wasted time ONLY with regards to the activity described by the ‘interval’ even (time difference between activities)

trace_log = interval_lifecycle.assign_lead_cycle_time(trace_log)

# Extracting target variable

- In our case target variable is REJECTED Declaration, in the process whoever rejects the Declaration (Administrartor, pre-approvar, etc.) it is considered as Declaration REJECTED so using it to identify our target variable

- One of our assumption is - If the decleration is not submitted by Employee and the payment is handeled directly, it is also considered as Declaration Accapted (so basically all the Declarations which is not rejected by employee are considerd as Accapted )

- we are not dealing with situations where it got rejected twice. For now we are only interested in 1st time rejection


In [481]:
# to extract target varaible, 
# if event starts with the name decleration rejected it is considered as rejected
declerations = []
for trace in trace_log:
    flag = False
    for i,event in enumerate(trace):
        if "Declaration REJECTED" in event['event']:
            flag = True
            break
    
    if flag:
        declerations.append(1)
    else:
        declerations.append(0)

In [482]:
print(len(declerations))

5596


# Extracting Prefix 

- No bucketing/clustering/binding is applied (to do if needed)
- we extracted traces before the rejection for decleration, because we are intrested in prediction for the traces before that only 
- some process do not follow the above path but those are considered as accapted
- Trace length ( 10 for now - no specific reason, process is not big so randomly chosen 10 / to do)
- so basically all the traces of length less than equal to 10 are chosen, 
    - We need to deal if vector length is not same (10 in our case) - we performed padding (adding zeros at the end) during encoding for those traces
    - so we have traces with length 1,2,... 10 
    - any trace with length less then equal to 10 can be predicted using our model

In [483]:
# to extract only traces till the point decleration is rejected
prefix_traces = []
for trace in trace_log:
    for i,event in enumerate(trace):
        if "Declaration REJECTED" in event['event']:
            break
    prefix_traces.append(Trace(trace[:i], attributes = trace.attributes))
prefix_traces = EventLog(prefix_traces)

In [484]:
# check if all good
project_nth(prefix_traces, 3)

['Permit SUBMITTED by EMPLOYEE', 'Permit APPROVED by ADMINISTRATION', 'Start trip', 'End trip', 'Permit FINAL_APPROVED by SUPERVISOR', 'Declaration SUBMITTED by EMPLOYEE', 'Declaration APPROVED by ADMINISTRATION', 'Declaration FINAL_APPROVED by SUPERVISOR', 'Request Payment']


In [485]:
# generate prefixes, note that we need to add the casts to EventLog and Trace to make sure that the result is a PM4Py EventLog object
trace_prefixes = EventLog([Trace(trace[0:t_length], attributes = trace.attributes) for trace in trace_log])

In [486]:
# check the trace length
print([len(trace) for trace in trace_log][0:15])
print([len(trace) for trace in trace_prefixes][0:15])

[18, 18, 8, 10, 10, 13, 19, 7, 10, 12, 8, 12, 7, 8, 7]
[4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4]


In [487]:
# check if all good
project_nth(trace_prefixes, 98)

['Permit SUBMITTED by EMPLOYEE', 'Permit APPROVED by ADMINISTRATION', 'Start trip', 'Permit FINAL_APPROVED by SUPERVISOR']


In [488]:
# to check which traces are not of our desired length (these will be padded while preparing the the training data)
for i,trace in enumerate(trace_prefixes):
    if len(trace)!=t_length:
        print(i, len(trace))
        break

In [489]:
# convert logs to dataframe
# final base dataframe
df = pm4py.convert_to_dataframe(trace_prefixes)
df.head(5)

,case,event,startTime,completeTime,id,RequestedAmount_0,Overspent,travel permit number,OverspentAmount,RequestedBudget,...,year,month,concept:name,time:timestamp,@@approx_bh_partial_cycle_time,@@approx_bh_partial_lead_time,@@approx_bh_overall_wasted_time,@@approx_bh_this_wasted_time,@approx_bh_ratio_cycle_lead_time,case:concept:name
0,travel permit 10022,Permit SUBMITTED by EMPLOYEE,2018-02-20 13:51:27+00:00,2018-02-20 13:51:27+00:00,travel permit 10022,395,False,travel permit number 10023,1969-12-31 23:59:59.999999950+00:00,1970-01-01 00:00:00.000002134+00:00,...,2018,2,Permit SUBMITTED by EMPLOYEE,2018-02-20 13:51:27+00:00,0.0,0.0,0.0,0.0,1.0,travel permit 10022
1,travel permit 10022,Permit APPROVED by ADMINISTRATION,2018-02-20 13:51:34+00:00,2018-02-20 13:51:34+00:00,travel permit 10022,395,False,travel permit number 10023,1969-12-31 23:59:59.999999950+00:00,1970-01-01 00:00:00.000002134+00:00,...,2018,2,Permit APPROVED by ADMINISTRATION,2018-02-20 13:51:34+00:00,0.0,7.0,7.0,7.0,0.0,travel permit 10022
2,travel permit 10022,Permit APPROVED by BUDGET OWNER,2018-02-20 16:27:33+00:00,2018-02-20 16:27:33+00:00,travel permit 10022,395,False,travel permit number 10023,1969-12-31 23:59:59.999999950+00:00,1970-01-01 00:00:00.000002134+00:00,...,2018,2,Permit APPROVED by BUDGET OWNER,2018-02-20 16:27:33+00:00,0.0,9366.0,9366.0,9359.0,0.0,travel permit 10022
3,travel permit 10022,Permit FINAL_APPROVED by SUPERVISOR,2018-02-21 12:58:49+00:00,2018-02-21 12:58:49+00:00,travel permit 10022,395,False,travel permit number 10023,1969-12-31 23:59:59.999999950+00:00,1970-01-01 00:00:00.000002134+00:00,...,2018,2,Permit FINAL_APPROVED by SUPERVISOR,2018-02-21 12:58:49+00:00,0.0,32842.0,32842.0,23476.0,0.0,travel permit 10022
4,travel permit 10040,Permit SUBMITTED by EMPLOYEE,2018-02-20 13:53:11+00:00,2018-02-20 13:53:11+00:00,travel permit 10040,2129.846048886792,False,travel permit number 10041,1969-12-31 23:59:59.999999354+00:00,1970-01-01 00:00:00.000002531+00:00,...,2018,2,Permit SUBMITTED by EMPLOYEE,2018-02-20 13:53:11+00:00,0.0,0.0,0.0,0.0,1.0,travel permit 10040


## Feature Selection ( Based on Data Exploration )

In [490]:
# passed features we want to extract

# str_ev_attr	String attributes at the event level: these are hot-encoded into features that may assume value 0 or value 1.
# str_tr_attr	String attributes at the trace level: these are hot-encoded into features that may assume value 0 or value 1.
# num_ev_attr	Numeric attributes at the event level: these are encoded by including the last value of the attribute among the events of the trace.
# num_tr_attr	Numeric attributes at trace level: these are encoded by including the numerical value.
# str_evsucc_attr	Successions related to the string attributes values at the event level: for example, if we have a trace [A,B,C], it might be important to include not only the presence of the single values A, B and C as features; but also the presence of the directly-follows couples (A,B) and (B,C).
# ================================================

str_ev_attr = ['concept:name']
str_tr_attr = ['OrganizationalEntity','month']
num_ev_attr = ['@@approx_bh_partial_lead_time','@@approx_bh_this_wasted_time']
num_tr_attr = []

# Data encoding

- preparing input data to be passed, 
- there are different encodings we can go with
    - boolean encoding (one hot encoding if the activity present or not)
    - frequency encoding (count of activity)
    - simple index encoding (n events one hot encoding)
    - latest payload encoding (with trace attributes)
    - index payload encoding (n events one hot encoding + with trace attributes)
    - complex index based encoding (static feature + nevents encoding + event features )
    - lstm encoding (m x n)
    
=========  

- Since the complex index based encoding covers the above 3 ( we can simply filter columns to get those, also feature importance (random forest) can ignore those which are not imporant ) we basically generated 4 different encodings -
    - boolean encoding (one hot encoding if the activity present or not)
    - frequency encoding (count of activity)
    - complex index based encoding (static feature + nevents encoding + event features )
    - lstm encoding (m x n)

=======
- No need of paddings in boolean and frequency encoding as the activities are fixed \

**PADDING** 
- for one hot encoded attributes (categorical values) the desired vector length is  - t_length * ohe_length (Ex. 10*number of activities) 
- for numerical values the desired length is t_length

**Note** - For now process changes are not been considered, if a new activity arrives in future

## Common functions for Encodings 

In [491]:
# function to save the data
def save_data(X,y,feature_names, save_path):
    data_dict = {}
    data_dict['X'] = X
    data_dict['y'] = y
    data_dict['feature_names'] = feature_names

    # save pickle
    with open(save_path, 'wb') as handle:
        pickle.dump(data_dict, handle)

In [492]:
# load the data from pickle
def load_data(load_path):
    with open(load_path, 'rb') as handle:
        data = pickle.load(handle)
    return data

In [493]:
# function to get the one hot encoded vectors of categorical values
def get_ohe_dict(categorical_vars, df):
    ohe_dict = {}
    for var in categorical_vars:
        var_dict = {}
        var_data = sorted(df[var].unique())
        var_len = len(var_data)
        for i,cat in enumerate(var_data):
            var_dict[cat] = [0]*var_len
            var_dict[cat][i] = 1

        ohe_dict[var] = var_dict

    return ohe_dict

In [494]:
# padding function for ohe encoding
def cat_padding(vec, t_length, attr_length):
    desired_length = t_length*attr_length
    vec_length = len(vec)
    if vec_length != desired_length:
        pad_vec = [0]*(desired_length-vec_length)
        vec.extend(pad_vec)
    return vec

In [495]:
# padding function for non-ohe encoding
def num_padding(vec, t_length):
    vec_length = len(vec)
    if vec_length != t_length:
        pad_vec = [0]*(t_length-vec_length)
        vec.extend(pad_vec)
    return vec

In [496]:
# create one hot encoding dict fot categorical variables
# variables which we want to be one hot encoded
categorical_vars = str_ev_attr + str_tr_attr
ohe_dict = get_ohe_dict(categorical_vars, df)
categorical_vars

['concept:name', 'OrganizationalEntity', 'month']

## Boolean encoding

- can be easily extended with adding more attributes of traces other then activities, but for now kept it simple and as discussed in course

In [497]:
# here for each trace we extract ohe vector for activity and sum them up and if count is greater then 1 we make them 1 
# because this encoding only provides info, if the activity was there or not

data = []

for id, group in df.groupby(['case:concept:name']):
    feature_vec = []

    # add categorical and numerical event attributes
    for cat_atr in str_ev_attr[:1]:
        attr_length = len(list(ohe_dict[cat_atr].values())[0])
        str_ev_vec = np.array([0]*attr_length)

        for ca in group[cat_atr]:
            str_ev_vec  = str_ev_vec + np.array(ohe_dict[cat_atr][ca])

        # make it a non frequency vector (if count is greater then 1 make it 1)
        for i,num in enumerate(str_ev_vec):
            if num>1:
                str_ev_vec[i]=1

        feature_vec.extend(list(str_ev_vec))

    data.append(feature_vec)

    # data.append(feature_vec)

In [498]:
# saving data 
encode_name = 'boolean_encode_'
save_path = save_path_base + encode_name + df_type +'_trace_len_'+str(t_length)+ '.pickle'
save_data(data, declerations, ohe_dict ,save_path)

## Frequency encoding

In [499]:
# here for each trace we extract ohe vector for activity and sum them up 
# because this encoding only provides count of how many times the activity appears

data = []

for id, group in df.groupby(['case:concept:name']):
    feature_vec = []

    # add categorical and numerical event attributes
    for cat_atr in str_ev_attr[:1]:
        attr_length = len(list(ohe_dict[cat_atr].values())[0])
        str_ev_vec = np.array([0]*attr_length)

        for ca in group[cat_atr]:
            str_ev_vec  = str_ev_vec + np.array(ohe_dict[cat_atr][ca])

        feature_vec.extend(list(str_ev_vec))

    data.append(feature_vec)

    # data.append(feature_vec)

In [500]:
# save results
encode_name = 'frequency_encode_'
save_path = save_path_base + encode_name + df_type +'_trace_len_'+str(t_length)+ '.pickle'
save_data(data, declerations, ohe_dict ,save_path)

## Complex index based encoding - static feature (trace attributes) + n events encoding + event features

In [501]:
# here for each trace we put events encoded in order and there aatributes along with padding to make vector length same
# similatly for trace attributes but since it is trace attributes that is only for once

data = []

for id, group in df.groupby(['case:concept:name']):
    feature_vec = []

    # add categorical and numerical event attributes along with paddings 

    for cat_atr in str_ev_attr:
        str_ev_vec = []
        attr_length = len(list(ohe_dict[cat_atr].values())[0])
        for ca in group[cat_atr]:
            str_ev_vec.extend(ohe_dict[cat_atr][ca])
        
        # padding
        str_ev_vec = cat_padding(str_ev_vec, t_length, attr_length)
        feature_vec.extend(str_ev_vec)


    for num_atr in num_ev_attr:
        num_ev_vec = []
        num_ev_vec.extend(list(group[num_atr]))

        # padding
        num_ev_vec = num_padding(num_ev_vec, t_length)
        feature_vec.extend(num_ev_vec)



    # add categorical and numerical trace attributes
    for num_t_atr in num_tr_attr:
        feature_vec.extend(group[num_t_atr].iloc[0])


    for cat_t_atr in str_tr_attr:
        feature_vec.extend(ohe_dict[cat_t_atr][group[cat_t_atr].iloc[0]])

    # add vector to data
    data.append(feature_vec)


In [502]:
# check if all vector lengths are same 
vec_len = len(data[0])
for i, d in enumerate(data):
    if len(d)!=vec_len:
        print(i, len(d))

In [503]:
# save results
encode_name = 'complex_index_encode_'
save_path = save_path_base + encode_name + df_type +'_trace_len_'+str(t_length)+ '.pickle'
save_data(data, declerations, ohe_dict ,save_path)

## LSTM encoding

In [504]:
# here we create sequence of each trace
# so the dimentions will be (number of examples * trace_length * feature_length )

data = []

for id, group in df.groupby(['case:concept:name']):
    
    feature_vec = []

    for index, row in group.iterrows():

        row_vec = []

        for cat_atr in str_ev_attr:
            row_vec.extend(ohe_dict[cat_atr][row[cat_atr]])

        for num_atr in num_ev_attr:
            row_vec.append(row[num_atr])


        # add categorical and numerical trace attributes
        for num_t_atr in num_tr_attr:
            row_vec.append(group[num_t_atr].iloc[0])

        for cat_t_atr in str_tr_attr:
            row_vec.extend(ohe_dict[cat_t_atr][group[cat_t_atr].iloc[0]])

        feature_vec.append(row_vec)
    

    # add vector to data
    data.append(feature_vec)


In [505]:
# converting to array
data = np.array([np.array(ls) for ls in data])

In [506]:
# shape we want for all the traces
feature_len = len(data[0][0])
desired_shape = (t_length,feature_len)
desired_shape

(4, 67)

In [507]:
# padding data to make equal shape of vectors
padded_data = []
for case in data:
    pd_case = np.zeros(desired_shape)
    pd_case[:case.shape[0],:case.shape[1]] = case
    padded_data.append(pd_case)

padded_data = np.array(padded_data)
padded_data.shape

(5596, 4, 67)

In [508]:
# save results
encode_name = 'lstm_encode_'
save_path = save_path_base + encode_name + df_type +'_trace_len_'+str(t_length)+ '.pickle'
save_data(padded_data, declerations, ohe_dict ,save_path)

# =============================================
# Preparing Test Data

- exactly same, just using **train encoding** to create features
- define trace length and df_type (train or test)
- Working on train and test seperately, so run whole code for the same trace length for train and then for test

In [509]:
df_type = 'test'
permits = test_df

In [510]:
test_df.head()

,case,event,startTime,completeTime,id,RequestedAmount_0,Overspent,travel permit number,OverspentAmount,RequestedBudget,...,event_id,org:role,@@index,@@case_index,start_timestamp,year,month,case:concept:name,concept:name,time:timestamp
1168,travel permit 11247,Permit SUBMITTED by EMPLOYEE,2018-10-24 11:45:31+00:00,2018-10-24 11:45:31+00:00,travel permit 11247,48,False,travel permit number 11248,1970-01-01 00:00:00+00:00,1970-01-01 00:00:00.000000061+00:00,...,st_step 11252_0,EMPLOYEE,1970-01-01 00:00:00.000001335+00:00,1970-01-01 00:00:00.000000106+00:00,2018-10-24 11:45:31+00:00,2018,10,travel permit 11247,Permit SUBMITTED by EMPLOYEE,2018-10-24 11:45:31+00:00
1169,travel permit 11247,Permit APPROVED by ADMINISTRATION,2018-10-24 11:45:35+00:00,2018-10-24 11:45:35+00:00,travel permit 11247,48,False,travel permit number 11248,1970-01-01 00:00:00+00:00,1970-01-01 00:00:00.000000061+00:00,...,st_step 11254_0,ADMINISTRATION,1970-01-01 00:00:00.000001336+00:00,1970-01-01 00:00:00.000000106+00:00,2018-10-24 11:45:35+00:00,2018,10,travel permit 11247,Permit APPROVED by ADMINISTRATION,2018-10-24 11:45:35+00:00
1170,travel permit 11247,Permit APPROVED by BUDGET OWNER,2018-10-25 08:31:16+00:00,2018-10-25 08:31:16+00:00,travel permit 11247,48,False,travel permit number 11248,1970-01-01 00:00:00+00:00,1970-01-01 00:00:00.000000061+00:00,...,st_step 11253_0,BUDGET OWNER,1970-01-01 00:00:00.000001337+00:00,1970-01-01 00:00:00.000000106+00:00,2018-10-25 08:31:16+00:00,2018,10,travel permit 11247,Permit APPROVED by BUDGET OWNER,2018-10-25 08:31:16+00:00
1171,travel permit 11247,Permit FINAL_APPROVED by SUPERVISOR,2018-10-30 07:38:35+00:00,2018-10-30 07:38:35+00:00,travel permit 11247,48,False,travel permit number 11248,1970-01-01 00:00:00+00:00,1970-01-01 00:00:00.000000061+00:00,...,st_step 11255_0,SUPERVISOR,1970-01-01 00:00:00.000001338+00:00,1970-01-01 00:00:00.000000106+00:00,2018-10-30 07:38:35+00:00,2018,10,travel permit 11247,Permit FINAL_APPROVED by SUPERVISOR,2018-10-30 07:38:35+00:00
1172,travel permit 11247,Declaration SUBMITTED by EMPLOYEE,2018-11-27 17:36:46+00:00,2018-11-27 17:36:46+00:00,travel permit 11247,48,False,travel permit number 11248,1970-01-01 00:00:00+00:00,1970-01-01 00:00:00.000000061+00:00,...,st_step 11257_0,EMPLOYEE,1970-01-01 00:00:00.000001339+00:00,1970-01-01 00:00:00.000000106+00:00,2018-11-27 17:36:46+00:00,2018,11,travel permit 11247,Declaration SUBMITTED by EMPLOYEE,2018-11-27 17:36:46+00:00


### Adding some stastitics features in event log using pm4py

In [511]:
# adding month and year as feature
permits['startTime'] = pd.to_datetime(permits['startTime'])
permits['completeTime'] = pd.to_datetime(permits['completeTime'])
permits['year'] = permits['startTime'].dt.year
permits['month'] = permits['startTime'].dt.month

In [512]:
# converting dataframe to event log
trace_log = pm4py.format_dataframe(permits, case_id='case', activity_key='event', timestamp_key='completeTime', start_timestamp_key='startTime')
trace_log = pm4py.convert_to_event_log(trace_log)

In [513]:
# adding lead and waiting time
# @@approx_bh_partial_lead_time	- Incremental lead time associated to the event
# @@approx_bh_overall_wasted_time - Difference between the partial lead time and the partial cycle time values (for us same as above, as no cylce time for us)
# @@approx_bh_this_wasted_time -	Wasted time ONLY with regards to the activity described by the ‘interval’ even (time difference between activities)

trace_log = interval_lifecycle.assign_lead_cycle_time(trace_log)

# Extracting target variable

- In our case target variable is REJECTED Declaration, in the process whoever rejects the Declaration (Administrartor, pre-approvar, etc.) it is considered as Declaration REJECTED so using it to identify our target variable

- One of our assumption is - If the decleration is not submitted by Employee and the payment is handeled directly, it is also considered as Declaration Accapted (so basically all the Declarations which is not rejected by employee are considerd as Accapted )

- we are not dealing with situations where it got rejected twice. For now we are only interested in 1st time rejection


In [514]:
# to extract target varaible, 
# if event starts with the name decleration rejected it is considered as rejected
declerations = []
for trace in trace_log:
    flag = False
    for i,event in enumerate(trace):
        if "Declaration REJECTED" in event['event']:
            flag = True
            break
    
    if flag:
        declerations.append(1)
    else:
        declerations.append(0)

In [515]:
print(len(declerations))

723


# Extracting Prefix 

- No bucketing/clustering/binding is applied (to do if needed)
- we extracted traces before the rejection for decleration, because we are intrested in prediction for the traces before that only 
- some process do not follow the above path but those are considered as accapted
- Trace length ( 10 for now - no specific reason, process is not big so randomly chosen 10 / to do)
- so basically all the traces of length less than equal to 10 are chosen, 
    - We need to deal if vector length is not same (10 in our case) - we performed padding (adding zeros at the end) during encoding for those traces
    - so we have traces with length 1,2,... 10 
    - any trace with length less then equal to 10 can be predicted using our model

In [516]:
# to extract only traces till the point decleration is rejected
prefix_traces = []
for trace in trace_log:
    for i,event in enumerate(trace):
        if "Declaration REJECTED" in event['event']:
            break
    prefix_traces.append(Trace(trace[:i], attributes = trace.attributes))
prefix_traces = EventLog(prefix_traces)

In [517]:
# check if all good
project_nth(prefix_traces, 3)

['Permit SUBMITTED by EMPLOYEE', 'Permit APPROVED by ADMINISTRATION', 'Permit FINAL_APPROVED by SUPERVISOR', 'Start trip', 'End trip', 'Declaration SUBMITTED by EMPLOYEE', 'Declaration APPROVED by ADMINISTRATION', 'Declaration FINAL_APPROVED by SUPERVISOR', 'Request Payment']


In [518]:
# generate prefixes, note that we need to add the casts to EventLog and Trace to make sure that the result is a PM4Py EventLog object
trace_prefixes = EventLog([Trace(trace[0:t_length], attributes = trace.attributes) for trace in trace_log])

In [519]:
# check the trace length
print([len(trace) for trace in trace_log][0:15])
print([len(trace) for trace in trace_prefixes][0:15])

[12, 12, 7, 10, 12, 5, 10, 6, 10, 13, 14, 10, 14, 5, 5]
[4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4]


In [520]:
# check if all good
project_nth(trace_prefixes, 98)

['Permit SUBMITTED by EMPLOYEE', 'Permit APPROVED by ADMINISTRATION', 'Permit FINAL_APPROVED by SUPERVISOR', 'Start trip']


In [521]:
# to check which traces are not of our desired length (these will be padded while preparing the the training data)
for i,trace in enumerate(trace_prefixes):
    if len(trace)!=t_length:
        print(i, len(trace))
        break

In [522]:
# convert logs to dataframe
# final base dataframe
df = pm4py.convert_to_dataframe(trace_prefixes)
df.head(5)

,case,event,startTime,completeTime,id,RequestedAmount_0,Overspent,travel permit number,OverspentAmount,RequestedBudget,...,year,month,concept:name,time:timestamp,@@approx_bh_partial_cycle_time,@@approx_bh_partial_lead_time,@@approx_bh_overall_wasted_time,@@approx_bh_this_wasted_time,@approx_bh_ratio_cycle_lead_time,case:concept:name
0,travel permit 11247,Permit SUBMITTED by EMPLOYEE,2018-10-24 11:45:31+00:00,2018-10-24 11:45:31+00:00,travel permit 11247,48,False,travel permit number 11248,1970-01-01 00:00:00+00:00,1970-01-01 00:00:00.000000061+00:00,...,2018,10,Permit SUBMITTED by EMPLOYEE,2018-10-24 11:45:31+00:00,0.0,0.0,0.0,0.0,1.0,travel permit 11247
1,travel permit 11247,Permit APPROVED by ADMINISTRATION,2018-10-24 11:45:35+00:00,2018-10-24 11:45:35+00:00,travel permit 11247,48,False,travel permit number 11248,1970-01-01 00:00:00+00:00,1970-01-01 00:00:00.000000061+00:00,...,2018,10,Permit APPROVED by ADMINISTRATION,2018-10-24 11:45:35+00:00,0.0,4.0,4.0,4.0,0.0,travel permit 11247
2,travel permit 11247,Permit APPROVED by BUDGET OWNER,2018-10-25 08:31:16+00:00,2018-10-25 08:31:16+00:00,travel permit 11247,48,False,travel permit number 11248,1970-01-01 00:00:00+00:00,1970-01-01 00:00:00.000000061+00:00,...,2018,10,Permit APPROVED by BUDGET OWNER,2018-10-25 08:31:16+00:00,0.0,24345.0,24345.0,24341.0,0.0,travel permit 11247
3,travel permit 11247,Permit FINAL_APPROVED by SUPERVISOR,2018-10-30 07:38:35+00:00,2018-10-30 07:38:35+00:00,travel permit 11247,48,False,travel permit number 11248,1970-01-01 00:00:00+00:00,1970-01-01 00:00:00.000000061+00:00,...,2018,10,Permit FINAL_APPROVED by SUPERVISOR,2018-10-30 07:38:35+00:00,0.0,129184.0,129184.0,104839.0,0.0,travel permit 11247
4,travel permit 11260,Permit SUBMITTED by EMPLOYEE,2018-10-24 12:38:03+00:00,2018-10-24 12:38:03+00:00,travel permit 11260,44,False,travel permit number 11261,1969-12-31 23:59:59.999999939+00:00,1970-01-01 00:00:00.000000059+00:00,...,2018,10,Permit SUBMITTED by EMPLOYEE,2018-10-24 12:38:03+00:00,0.0,0.0,0.0,0.0,1.0,travel permit 11260


## Feature Selection ( Based on Data Exploration )

In [523]:
# passed features we want to extract

# str_ev_attr	String attributes at the event level: these are hot-encoded into features that may assume value 0 or value 1.
# str_tr_attr	String attributes at the trace level: these are hot-encoded into features that may assume value 0 or value 1.
# num_ev_attr	Numeric attributes at the event level: these are encoded by including the last value of the attribute among the events of the trace.
# num_tr_attr	Numeric attributes at trace level: these are encoded by including the numerical value.
# str_evsucc_attr	Successions related to the string attributes values at the event level: for example, if we have a trace [A,B,C], it might be important to include not only the presence of the single values A, B and C as features; but also the presence of the directly-follows couples (A,B) and (B,C).
# ================================================

str_ev_attr = ['concept:name']
str_tr_attr = ['OrganizationalEntity','month']
num_ev_attr = ['@@approx_bh_partial_lead_time','@@approx_bh_this_wasted_time']
num_tr_attr = []

## Boolean encoding

- can be easily extended with adding more attributes of traces other then activities, but for now kept it simple and as discussed in course

In [524]:
encode_name = 'boolean_encode_'
save_path = save_path_base + encode_name + 'train' +'_trace_len_'+str(t_length)+ '.pickle'
loaded_data = load_data(save_path)
ohe_dict = loaded_data['feature_names']

In [525]:
# here for each trace we extract ohe vector for activity and sum them up and if count is greater then 1 we make them 1 
# because this encoding only provides info, if the activity was there or not

data = []

for id, group in df.groupby(['case:concept:name']):
    feature_vec = []

    # add categorical and numerical event attributes
    for cat_atr in str_ev_attr[:1]:
        attr_length = len(list(ohe_dict[cat_atr].values())[0])
        str_ev_vec = np.array([0]*attr_length)

        for ca in group[cat_atr]:
            str_ev_vec  = str_ev_vec + np.array(ohe_dict[cat_atr][ca])

        # make it a non frequency vector (if count is greater then 1 make it 1)
        for i,num in enumerate(str_ev_vec):
            if num>1:
                str_ev_vec[i]=1

        feature_vec.extend(list(str_ev_vec))

    data.append(feature_vec)

    # data.append(feature_vec)

In [526]:
encode_name = 'boolean_encode_'
save_path = save_path_base + encode_name + df_type +'_trace_len_'+str(t_length)+ '.pickle'
save_data(data, declerations, ohe_dict ,save_path)

## Frequency encoding

In [527]:
encode_name = 'frequency_encode_'
save_path = save_path_base + encode_name + 'train' +'_trace_len_'+str(t_length)+ '.pickle'
loaded_data = load_data(save_path)
ohe_dict = loaded_data['feature_names']

In [528]:
# here for each trace we extract ohe vector for activity and sum them up 
# because this encoding only provides count of how many times the activity appears

data = []

for id, group in df.groupby(['case:concept:name']):
    feature_vec = []

    # add categorical and numerical event attributes
    for cat_atr in str_ev_attr[:1]:
        attr_length = len(list(ohe_dict[cat_atr].values())[0])
        str_ev_vec = np.array([0]*attr_length)

        for ca in group[cat_atr]:
            str_ev_vec  = str_ev_vec + np.array(ohe_dict[cat_atr][ca])

        feature_vec.extend(list(str_ev_vec))

    data.append(feature_vec)

    # data.append(feature_vec)

In [529]:
# save results
encode_name = 'frequency_encode_'
save_path = save_path_base + encode_name + df_type +'_trace_len_'+str(t_length)+ '.pickle'
save_data(data, declerations, ohe_dict ,save_path)

## Complex index based encoding - static feature (trace attributes) + n events encoding + event features

In [530]:
encode_name = 'complex_index_encode_'
save_path = save_path_base + encode_name + 'train' +'_trace_len_'+str(t_length)+ '.pickle'
loaded_data = load_data(save_path)
ohe_dict = loaded_data['feature_names']

In [531]:
# here for each trace we put events encoded in order and there aatributes along with padding to make vector length same
# similatly for trace attributes but since it is trace attributes that is only for once

data = []

for id, group in df.groupby(['case:concept:name']):
    feature_vec = []

    # add categorical and numerical event attributes along with paddings 

    for cat_atr in str_ev_attr:
        str_ev_vec = []
        attr_length = len(list(ohe_dict[cat_atr].values())[0])
        for ca in group[cat_atr]:
            str_ev_vec.extend(ohe_dict[cat_atr][ca])
        
        # padding
        str_ev_vec = cat_padding(str_ev_vec, t_length, attr_length)
        feature_vec.extend(str_ev_vec)


    for num_atr in num_ev_attr:
        num_ev_vec = []
        num_ev_vec.extend(list(group[num_atr]))

        # padding
        num_ev_vec = num_padding(num_ev_vec, t_length)
        feature_vec.extend(num_ev_vec)



    # add categorical and numerical trace attributes
    for num_t_atr in num_tr_attr:
        feature_vec.extend(group[num_t_atr].iloc[0])


    for cat_t_atr in str_tr_attr:
        feature_vec.extend(ohe_dict[cat_t_atr][group[cat_t_atr].iloc[0]])

    # add vector to data
    data.append(feature_vec)


In [532]:
# check if all vector lengths are same 
vec_len = len(data[0])

for i, d in enumerate(data):
    if len(d)!=vec_len:
        print(i, len(d))

In [533]:
# save results
encode_name = 'complex_index_encode_'
save_path = save_path_base + encode_name + df_type +'_trace_len_'+str(t_length)+ '.pickle'
save_data(data, declerations, ohe_dict ,save_path)

## LSTM encoding

In [534]:
encode_name = 'lstm_encode_'
save_path = save_path_base + encode_name + 'train' +'_trace_len_'+str(t_length)+ '.pickle'
loaded_data = load_data(save_path)
ohe_dict = loaded_data['feature_names']

In [535]:
# here we create sequence of each trace
# so the dimentions will be (number of examples * trace_length * feature_length )

data = []

for id, group in df.groupby(['case:concept:name']):
    
    feature_vec = []

    for index, row in group.iterrows():

        row_vec = []

        for cat_atr in str_ev_attr:
            row_vec.extend(ohe_dict[cat_atr][row[cat_atr]])

        for num_atr in num_ev_attr:
            row_vec.append(row[num_atr])


        # add categorical and numerical trace attributes
        for num_t_atr in num_tr_attr:
            row_vec.append(group[num_t_atr].iloc[0])

        for cat_t_atr in str_tr_attr:
            row_vec.extend(ohe_dict[cat_t_atr][group[cat_t_atr].iloc[0]])

        feature_vec.append(row_vec)
    

    # add vector to data
    data.append(feature_vec)


In [536]:
# converting to array
data = np.array([np.array(ls) for ls in data])

In [537]:
feature_len = len(data[0][0])
desired_shape = (t_length,feature_len)
desired_shape

(4, 67)

In [538]:
# padding data to make equal shape of vectors
padded_data = []
for case in data:
    pd_case = np.zeros(desired_shape)
    pd_case[:case.shape[0],:case.shape[1]] = case
    padded_data.append(pd_case)

padded_data = np.array(padded_data)
padded_data.shape

(723, 4, 67)

In [539]:
# save results
encode_name = 'lstm_encode_'
save_path = save_path_base + encode_name + df_type +'_trace_len_'+str(t_length)+ '.pickle'
save_data(data, declerations, ohe_dict ,save_path)

# =================== END =================



# Extra

In [ ]:
permits.head()
permits['startTime'] = pd.to_datetime(permits['startTime'])
permits['completeTime'] = pd.to_datetime(permits['completeTime'])

In [ ]:
# add year and month column
permits['year'] = permits['startTime'].dt.year
permits['month'] = permits['startTime'].dt.month

In [ ]:
# time to complete the activity
permits['act_completionTime'] = permits['completeTime'] - permits['startTime']
permits['act_completionTime'] = permits['act_completionTime'].dt.total_seconds()

In [ ]:
# get minimum time for earch trace (when trace started)
temp_df = pd.DataFrame(permits.groupby(['case'])['startTime'].min())
temp_df['index'] = temp_df.index
temp_df.reset_index(drop=True, inplace=True)
temp_df.columns = ['startTime_min','case']

# merge with permits
permits = permits.merge(temp_df,on=['case'])

In [ ]:
# get max time for earch trace (when trace ended)
temp_df = pd.DataFrame(permits.groupby(['case'])['completeTime'].max())
temp_df['index'] = temp_df.index
temp_df.reset_index(drop=True, inplace=True)
temp_df.columns = ['completeTime_max','case']

# merge with permits
permits = permits.merge(temp_df,on=['case'])

In [ ]:
# calculate trace time
permits['processCompletionTime']=permits['completeTime_max']-permits['startTime_min']
permits['processCompletionTime'] = permits['processCompletionTime'].dt.total_seconds()

In [ ]:
# calculate time taken between activities and cumulative time
act_time_taken=[]
act_inc_time = []
for name,group in permits.groupby(['case'],sort=False):
    act_time_taken.append([name,pd.to_datetime(0) - pd.to_datetime(0)])
    act_inc_time.append([name,pd.to_datetime(0) - pd.to_datetime(0)])
    act_time=pd.to_datetime(0) - pd.to_datetime(0)
    for i in range(len(group)-1):
        st_time_1 = group.iloc[i]['startTime']
        st_time_2 = group.iloc[i+1]['startTime']
        diff = st_time_2-st_time_1
        act_time+=diff
        act_time_taken.append([name,diff])
        act_inc_time.append([name,act_time])

time_diff_df = pd.DataFrame(act_time_taken,columns=['case','act_time_diff'])
time_inc_df = pd.DataFrame(act_inc_time,columns=['case','act_inc_time'])

# add with permits
permits['next_act_time_diff'] = time_diff_df['act_time_diff']
permits['next_act_time_diff'] = permits['next_act_time_diff'].dt.total_seconds()

permits['act_inc_time'] = time_inc_df['act_inc_time']
permits['act_inc_time'] = permits['act_inc_time'].dt.total_seconds()

In [ ]:
permits.head()

,case,event,startTime,completeTime,OrganizationalEntity,TotalDeclared,Overspent,RequestedBudget,OverspentAmount,org:resource,year,month,act_completionTime,startTime_min,completeTime_max,processCompletionTime,next_act_time_diff,act_inc_time
0,travel permit 10066,Permit SUBMITTED by EMPLOYEE,2018-02-21 16:28:17,2018-02-21 16:28:17,organizational unit 65460,71.195831,False,64.878503,-18.925474,STAFF MEMBER,2018,2,0.0,2018-02-21 16:28:17,2018-03-01 17:31:26,694989.0,0.0,0.0
1,travel permit 10066,Permit APPROVED by ADMINISTRATION,2018-02-21 16:28:19,2018-02-21 16:28:19,organizational unit 65460,71.195831,False,64.878503,-18.925474,STAFF MEMBER,2018,2,0.0,2018-02-21 16:28:17,2018-03-01 17:31:26,694989.0,2.0,2.0
2,travel permit 10066,Start trip,2018-02-22 00:00:00,2018-02-22 00:00:00,organizational unit 65460,71.195831,False,64.878503,-18.925474,STAFF MEMBER,2018,2,0.0,2018-02-21 16:28:17,2018-03-01 17:31:26,694989.0,27101.0,27103.0
3,travel permit 10066,End trip,2018-02-22 00:00:00,2018-02-22 00:00:00,organizational unit 65460,71.195831,False,64.878503,-18.925474,STAFF MEMBER,2018,2,0.0,2018-02-21 16:28:17,2018-03-01 17:31:26,694989.0,0.0,27103.0
4,travel permit 10066,Permit FINAL_APPROVED by SUPERVISOR,2018-02-22 08:27:05,2018-02-22 08:27:05,organizational unit 65460,71.195831,False,64.878503,-18.925474,STAFF MEMBER,2018,2,0.0,2018-02-21 16:28:17,2018-03-01 17:31:26,694989.0,30425.0,57528.0


In [ ]:
# from pm4py.statistics.sojourn_time.log import get as soj_time_get
# soj_time = soj_time_get.apply(trace_log, parameters={soj_time_get.Parameters.TIMESTAMP_KEY: "time:timestamp", soj_time_get.Parameters.START_TIMESTAMP_KEY: "startTime"})
# print(soj_time)

In [ ]:
# # to get all trace cases
# for i, trace in enumerate(trace_log):
#     print(trace[0]['case'])

In [ ]:
# # throughput time (time to complete traces) all in seconds
# all_case_durations = pm4py.get_all_case_durations(trace_log)
# all_case_durations

In [ ]:
# # arrival between cases
# case_arrival_ratio = pm4py.get_case_arrival_average(trace_log)
# case_arrival_ratio/(60*60)

In [ ]:
# from pm4py.objects.log.util import interval_lifecycle
# enriched_log = interval_lifecycle.assign_lead_cycle_time(trace_log)
# enriched_log

### boolean bigram encoding

In [ ]:
data_2gram, feature_names = log_to_features.apply(trace_prefixes, 
                                                  parameters={"str_ev_attr": [], 
                                                        "str_tr_attr": [], 
                                                        "num_ev_attr": [], 
                                                        "num_tr_attr": [], 
                                                        "str_evsucc_attr": ["concept:name"]})
feature_names

In [ ]:
data_2gram = np.asarray(data_2gram)

In [ ]:
project_nth(trace_prefixes, 0)

In [ ]:
print(data_2gram[0])

In [ ]:
train_data = {}
train_data['X']=data_2gram
train_data['y'] = declerations
train_data['feature_names'] = feature_names

In [ ]:
# save json
save_path = '../data/training_data/bigram_boolean_encode.json'
with open(save_path, 'w', encoding='utf-8') as f:
    json.dump(train_data, f, ensure_ascii=False, indent=4)

In [ ]:
# between_log = pm4py.filter_between(dec_sub_traces, "Permit SUBMITTED by EMPLOYEE", "Declaration SUBMITTED by EMPLOYEE")

In [ ]:
# project_nth(between_log, 232)

In [ ]:
# we can get any attribute of trace using this
activities = pm4py.get_event_attribute_values(trace_log, "concept:name")
resources = pm4py.get_event_attribute_values(trace_log, "org:resource")
resources

In [ ]:
# remove Return Decleration event
# train_raw = pm4py.filter_event_attribute_values(trace_log, "concept:name", "Declaration REJECTED by EMPLOYEE", level = "event", retain=False)

In [ ]:
# to display most frequent feature
# # look at the unique vectors and their occurrence frequency/count
# # just to check which trace was most frequent, just analysis
# dist_features = np.unique(data, return_counts= True, axis = 0)
# print(dist_features)


# # display most freuent feature
# most_freq_feature = dist_features[0][np.argmax(dist_features[1])]
# most_freq_feature


# # order doesn't matter here
# for i in range(0, len(most_freq_feature)):
#     if most_freq_feature[i]!=0:
#         print(feature_names[i].split("@")[1], end=' , ')

In [ ]:
# # printing all activities in trace 
# for trace in trace_log[0]:
#     print(trace["concept:name"])

In [ ]:
# to extract only traces where Declaration SUBMITTED by EMPLOYEE is present
# dec_sub_traces = []
# for trace in trace_log:
#     if len(list(filter(lambda e: e["concept:name"] == "Declaration SUBMITTED by EMPLOYEE" ,trace))) > 0:
#         dec_sub_traces.append(Trace(trace, attributes = trace.attributes))

# dec_sub_traces = EventLog(dec_sub_traces)